In [12]:
import json
import requests
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
from pprint import pprint

import warnings
warnings.simplefilter('ignore')

from config import CONFIG

### Загрузка резюме

In [11]:
remote_link = 'https://remote-job.ru/search?search%5BsearchType%5D=resume&category=1&page='
links = []
for i in tqdm(range(1, 94)):
    current_link = remote_link + str(i)
    request = requests.get(current_link)
    soup = BeautifulSoup(request.text)
    vacs = soup.find_all('div', {'class': 'vacancy_item'})
    for vac in vacs:
        links.append(vac.find('a', href=True)['href'])

100%|██████████████████████████████████████████████████████████████████████████████████| 93/93 [00:17<00:00,  6.58it/s]


### Загрузка текстов резюме

In [51]:
site_name = 'https://remote-job.ru'
cv = []

for link in tqdm(links):
    current_link = site_name + link
    request = requests.get(current_link)
    soup = BeautifulSoup(request.text)
    cur_cv = dict()
    header = soup.find('div', {'class': 'panel-heading'})
    cur_cv['title'] = header.find('h2').text
    add_header_info = header.find_all('b')
    if add_header_info is not None:
        cur_cv['salary'] = add_header_info[0].text
        cur_cv['place'] = add_header_info[1].text
    
    cv_text = soup.find('div', {'class': 'panel-body'}).find('div', {'class': 'col-md-12'}).text
    cur_cv['text'] = (
        cv_text
        .replace('\n\n', 'NEW LINE')
        .replace('\n', '')
        .replace('NEW LINE', '\n')
    )
    
    cv.append(cur_cv)
    time.sleep(1)

100%|██████████████████████████████████████████████████████████████████████████████| 1855/1855 [37:07<00:00,  1.17s/it]


### Сохранение в файл

In [62]:
with open(f'{CONFIG.DATA_FOLDER}/remote_cvs.json', 'w', encoding='utf-8') as f:
    json.dump(cv, f, ensure_ascii=False, indent=4)